In [1]:
import os
import json
import requests 
import pandas as pd
import numpy as np
from datetime import datetime
from config import riot_api

In [2]:
now = datetime.now() # Get current datetime
date = now.date().strftime("%B %d, %Y")
ogtime = now.time().strftime("%H:%M:%S")

# region code is [{BR1:"Brazil",EUN1:"EU Nordic & East",EUW1:"EU West",JP1:"Japan",KR:"Korea",LA1:"Latin America North" 
# LA2:"Latin America South,NA1:"North America,OC1:"Oceania",PBE1:"Public Beta Environment,RU:"Russia",TR1:Turkey}]

region_code = input("Enter Your Region: ").upper() # Ask for region code

if region_code in ["NA1", "BR1", "LA1", "LA2", "OC1"]: # Get region
    region = "americas"
elif region_code in ["KR", "JP1"]:
    region = "asia"
else:
    region = "europe"

try:
    os.makedirs(f"Analysis/{region_code}/{date}") # Make new  directory
except:
    print("Today's directory has been created already")
    
# Free Champions Rotation

url = f"https://{region_code}.api.riotgames.com/lol/platform/v3/champion-rotations?api_key=" + riot_api
champ_list = requests.get(url).json()
with open(f'Analysis/{region_code}/{date}/champ_rotation.json', 'w') as file: 
    file.write(json.dumps(champ_list))

# Live Games

url = f"https://{region_code}.api.riotgames.com/lol/spectator/v4/featured-games?api_key=" + riot_api
live_games = requests.get(url)
statcode = live_games.status_code

if statcode == 200:
    live_games_status = "Region availible"
else:
    live_games_status ="Region unavailible"

print(f"{live_games_status}\n{live_games.status_code}\n{champ_list}") 


Enter Your Region: kr
Today's directory has been created already
Region availible
200
{'freeChampionIds': [17, 43, 56, 62, 67, 79, 85, 90, 133, 145, 147, 157, 201, 203, 245, 518], 'freeChampionIdsForNewPlayers': [222, 254, 427, 82, 131, 147, 54, 17, 18, 37], 'maxNewPlayerLevel': 10}


In [3]:
# Save Live Game Data

live_games = live_games.json()
live_games_df = pd.DataFrame(live_games['gameList'])
live_games_df.to_csv(f'Analysis/{region_code}/{date}/live_game_data({ogtime}).csv')
    
live_games_df

,gameId,mapId,gameMode,gameType,gameQueueConfigId,participants,observers,platformId,bannedChampions,gameStartTime,gameLength
0,5905991960,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 6, 'spell2Id': 4,...",{'encryptionKey': 'GW7U9fQL2PO1ZIm0ypAqh84hXDe...,KR,[],1651956768344,389
1,5905942942,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 3, 'spell2Id': 4,...",{'encryptionKey': 'V5wxm/M9ZnSdgWgsjgSdQ9jgwud...,KR,[],1651956825476,331
2,5906001935,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 3, 'spell2Id': 4,...",{'encryptionKey': '87/soqPS2hHEvROkAKBBUAI4l3c...,KR,[],1651956986938,170
3,5905939536,11,CLASSIC,MATCHED_GAME,420,"[{'teamId': 100, 'spell1Id': 12, 'spell2Id': 4...",{'encryptionKey': 'aFh7jfLPCebtsmTD4MgrSe4fLu5...,KR,"[{'championId': 154, 'teamId': 100, 'pickTurn'...",1651957048156,109
4,5906011657,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 6, 'spell2Id': 4,...",{'encryptionKey': 'h1fCWfoQYI6HDHcT8HFsGasa8Ez...,KR,[],1651957129772,27


In [4]:
name = input("Enter Your Summoner Name: ") # And Username
try:
    os.makedirs(f"Analysis/{region_code}/{date}/{name}") # Make new  directory
except:
    print("Summoners directory has already been created")

Enter Your Summoner Name: NoWay4u
Summoners directory has already been created


In [5]:
# Summoners Data 

def Summoner (region_code, name):    
    
    url = f"https://{region_code}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key=" + riot_api
    summoner_info_1 = requests.get(url).json()
    iden, puuid = summoner_info_1["id"], summoner_info_1["puuid"]
    
    Ranked_info_url = f"https://{region_code}.api.riotgames.com/lol/league/v4/entries/by-summoner/{iden}?api_key=" + riot_api
    Champ_Pool_Mastery_url = f"https://{region_code}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{iden}?api_key=" + riot_api

    Ranked_info = requests.get(Ranked_info_url).json()    
    Champ_Pool_Mastery = requests.get(Champ_Pool_Mastery_url).json()
    if live_games_status == "Region availible":
        url = f"https://{region_code}.api.riotgames.com/lol/spectator/v4/active-games/by-summoner/{iden}?api_key=" + riot_api
        in_game_info =  requests.get(url)
        if in_game_info.status_code == "200":
            in_game_info =  in_game_info.json()
            print("In Live Game")
        else:
            in_game_info = np.nan
            print("No Live Data")
    else:
        in_game_info = np.nan
        
    return [summoner_info_1, Ranked_info, Champ_Pool_Mastery, in_game_info]

summoner_info = Summoner(region_code, name)
Summoner_info, Ranked_info, Champ_Pool_Mastery = [summoner_info[0]], summoner_info[1], summoner_info[2]



No Live Data


In [6]:
Summoner_info_df = pd.DataFrame(Summoner_info)

Summoner_info_df.to_csv(f'Analysis/{region_code}/{date}/{name}/data({ogtime}).csv', index=False)

Summoner_info_df.head()

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,MkqsHgO5A8n0yXwnEmrlTamCdjjjelzDp6LyfjhDMcY63k...,d_Aa0VG7QnvIreN5xTM1j68Hv08ovHypasl52qHooduv-5...,dOKU0qlxQvnzreMg6M0N2gQM9B1V-WffPsJGPgos9aKfxQ...,NoWay4u,29,1651467740000,37


In [7]:
if not Ranked_info:
    print("Unranked this Season")
    Ranked_info_df = pd.DataFrame()    

else:
    Ranked_info_df = pd.DataFrame(Ranked_info)
    Ranked_info_df.to_csv(f'Analysis/{region_code}/{date}/{name}/ranked_data_({ogtime}).csv', index=False)


Ranked_info_df.head()

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,aab9b730-2078-3349-9884-3f28e02a0610,RANKED_SOLO_5x5,MASTER,I,MkqsHgO5A8n0yXwnEmrlTamCdjjjelzDp6LyfjhDMcY63k...,NoWay4u,43,62,25,False,False,True,False


In [8]:
print(f"{region_code}, {region}, {name}")
Champ_Pool_Mastery_df = pd.DataFrame(Champ_Pool_Mastery)
Champ_Pool_Mastery_df.to_csv(f'Analysis/{region_code}/{date}/{name}/champ_pool_({ogtime}).csv', index=False)

Champ_Pool_Mastery_df.head()

KR, asia, NoWay4u


,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId
0,166,3,12333,1651954325000,6333,267,True,0,MkqsHgO5A8n0yXwnEmrlTamCdjjjelzDp6LyfjhDMcY63k...
1,112,3,8967,1651951957000,2967,3633,True,0,MkqsHgO5A8n0yXwnEmrlTamCdjjjelzDp6LyfjhDMcY63k...
2,69,3,6199,1651956418000,199,6401,True,0,MkqsHgO5A8n0yXwnEmrlTamCdjjjelzDp6LyfjhDMcY63k...
3,157,2,5096,1651939166000,3296,904,False,0,MkqsHgO5A8n0yXwnEmrlTamCdjjjelzDp6LyfjhDMcY63k...
4,103,2,3943,1651949430000,2143,2057,False,0,MkqsHgO5A8n0yXwnEmrlTamCdjjjelzDp6LyfjhDMcY63k...


In [ ]:
def Games(puuid, region):
    
    last_20_id_url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key=" + riot_api
    last_20_id = requests.get(last_20_id_url).json()
    
    match_data = [(requests.get(f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match}?api_key=" + riot_api).json()) 
                       for match in last_20_id] 
    time_line = [(requests.get(f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match}/timeline?api_key=" + riot_api).json()) 
                       for match in last_20_id]
    
    return [last_20_id, match_data, time_line]

puuid = Summoner_info_df["puuid"][0]
Last_20_Games_Metadata = Games(puuid, region)
Last_20_Games_Metadata_df = pd.DataFrame(Last_20_Games_Metadata)
try:
    os.makedirs(f"Analysis/{region_code}/{date}/{name}/Last_20_({ogtime})") # Make new  directory
except: 
    print("Folder has been created")

for j in range(len(Last_20_Games_Metadata_df.columns)): # for each match store its data and live data in new json elements
    game_id = Last_20_Games_Metadata[0][j]
    with open(f'Analysis/{region_code}/{date}/{name}/Last_20_({ogtime})/{game_id}.json', 'w') as file: 
            file.write(json.dumps(Last_20_Games_Metadata[1][j]))
    with open(f'Analysis/{region_code}/{date}/{name}/Last_20_({ogtime})/{game_id}_timeline.json', 'w') as file:
            file.write(json.dumps(Last_20_Games_Metadata[2][j]))

Last_20_Games_Metadata_df.head()


In [ ]:
with open(f'Analysis/user_input.py', 'w') as file: # Write inputs to seperate file
    file.write("region_code ="+ f' "{region_code}"\n')
    file.write("region =" + f' "{region}"')
   